# CSC/XMM hostless catalog

In [1]:
import sys
sys.path.append('../')

In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np

import matplotlib.pyplot as plt
# plt.style.use('seaborn')

import seaborn as sns

from scripts.utils import set_mpl, data_path
from scripts.cross_match_scripts import cross_match_data_frames,  add_separation_columns
set_mpl()

%matplotlib inline
from astropy.table import Table

matplotlib settings set


matplotlib settings set
matplotlib settings set


***

In [3]:
ero_df = pd.read_pickle(data_path+'ERO_lhpv_03_23_sd01_a15_g14.pkl') 
ero_df.sample(5)

,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH
5529,SRGe J103831.6+555105,159.631634,55.851279,153.421030,52.680708,8.514138,16.169888,6.189377e-15,NaN,30.277956,NaN,4469.172363,0.0,0.0,0.0,2015,1580,159.633642,55.849990,NaN,46.413334,0.000007,6.268413e+08,6.270701e+08,0.0,0.0,0.006775,NaN,18.518860,159.631634,55.851279,-1,-1.0,0,-1,NaN,-1.0,-1,NaN,NaN,NaN,-1.000000,0,0,0,-1,NaN,NaN,NaN,NaN,NaN,NaN,0,1,1,1237657222559629592,0,7.572594,71090849781,3.713041,3.043862,4.594991,7.450604,135.985298,42.966917,4.762967e-15,5.507220e+19
6579,SRGe J104819.9+600304,162.083104,60.051005,146.864687,50.971483,7.974242,12.564727,5.113257e-15,1.466591e-15,24.230377,6.949790,4329.224121,0.0,0.0,0.0,6371,5297,162.085113,60.049716,3.464623,170.834198,0.000010,6.268413e+08,6.270701e+08,0.0,0.0,0.005597,0.001605,21.009466,162.083104,60.051005,-1,-1.0,0,-1,NaN,-1.0,-1,NaN,NaN,NaN,-1.000000,0,0,0,1,16.239,15.020,11.874,19.2,16.2,4.3,1,1,0,1237655369283208174,0,9.824847,55056868747,3.464623,2.850846,4.303614,6.978148,134.370630,47.212634,3.934850e-15,7.484219e+19
3522,SRGe J104918.7+600902,162.328092,60.150507,146.616315,50.990525,7.947564,30.379967,9.656474e-15,2.013797e-15,37.732990,7.868979,3569.851318,0.0,0.0,0.0,3538,2818,162.330101,60.149218,3.452324,50.160748,0.000007,6.268413e+08,6.270701e+08,0.0,0.0,0.010570,0.002204,14.639443,162.328092,60.150507,-1,-1.0,0,-1,NaN,-1.0,-1,NaN,NaN,NaN,-1.000000,0,0,0,2,16.458,15.758,12.632,17.1,9.5,2.5,2,1,1,1237653617471521223,0,5.880972,54699733295,3.452324,2.841308,4.289217,6.954803,134.438080,47.363368,7.431033e-15,7.473973e+19
3985,SRGe J102550.5+562650,156.460564,56.447105,154.259806,50.890381,5.920263,42.692238,8.536577e-15,1.447087e-15,63.043705,10.686920,6746.918457,0.0,0.0,0.0,3062,2440,156.462573,56.445816,2.507454,77.965538,0.000007,6.268413e+08,6.270701e+08,0.0,0.0,0.009344,0.001584,45.715595,156.460564,56.447105,-1,-1.0,0,-1,NaN,-1.0,-1,NaN,NaN,NaN,-1.000000,0,0,0,-1,NaN,NaN,NaN,NaN,NaN,NaN,0,0,-1,0,0,NaN,68701782724,2.507454,2.116534,3.195103,5.180740,133.496866,42.594251,6.569229e-15,6.364085e+19
1674,SRGe J104757.3+580920,161.988667,58.155501,149.158726,52.222564,4.816288,84.121796,1.768276e-14,2.387078e-15,82.951096,11.197957,4285.679199,0.0,0.0,0.0,1907,1496,161.990675,58.154212,1.978187,118.776894,0.000006,6.268413e+08,6.270701e+08,0.0,0.0,0.019355,0.002613,17.738142,161.988667,58.155501,-1,-1.0,0,-1,NaN,-1.0,-1,SDSS J104757.00+580919.1,1.131128,QSO,7.200601,1,0,0,0,15.609,15.101,12.627,25.7,16.5,2.4,1,1,1,1237658303816335669,9215543088982020096,7.197365,62061132796,1.978187,1.721856,2.599300,4.214667,135.820900,45.601552,1.360757e-14,8.844219e+19


In [4]:
# desi_lh.gz_pkl number of sources and area
false_dens_arcmin = 2418574 / (41.729 * 3600)
print(false_dens_arcmin)
false_dens_arcsec = false_dens_arcmin / 3600

print(f'{ false_dens_arcsec:.3f}')

FALSE_PROB = .03



def fsource_r(rho: float, f_thresh: float) -> float:
    """
    The function calculates the radius which corresponds to the probability `f_thresh`
    to FIND one or more false sources. 

    Args:
        rho (float): the density of the false sources (in arcmin^{-2}).
        f_thresh (float): probability to FIND one or more false sources.

    Returns:
        float: radius (in arcsec).
    """

    rho = rho / 3600 # arcmin^{-2} -> arcsec^{-2}
    pf_r = np.sqrt(-1 / (rho * np.pi) * np.log(1 - f_thresh))
    
    return pf_r




r_false = fsource_r(false_dens_arcmin, FALSE_PROB)

print(f'Радиус для {FALSE_PROB:.0%} вероятности найти ложный: {r_false:.2f}"')

desi = pd.read_pickle(data_path+'desi_lh.gz_pkl', compression='gzip')
desi.sample(5)

16.099741453452303
0.004
Радиус для 3% вероятности найти ложный: 1.47"


,release,objid,brickid,ra,dec,flux_g,flux_r,flux_z,flux_w1,flux_w2,flux_w3,flux_w4,flux_ivar_g,flux_ivar_r,flux_ivar_z,flux_ivar_w1,flux_ivar_w2,flux_ivar_w3,flux_ivar_w4,dered_mag_g,dered_mag_r,dered_mag_z,dered_mag_w1,dered_mag_w2,dered_mag_w3,dered_mag_w4,snr_g,snr_r,snr_z,snr_w1,snr_w2,snr_w3,snr_w4,type,parallax,parallax_ivar,pmra,pmra_ivar,pmdec,pmdec_ivar,ref_cat,ref_id,mjd_max,mjd_min,iso_max,iso_min,desi_id
531073,9011,421,613761,157.962566,58.758743,0.675485,2.966796,5.946714,12.178886,9.764304,4.833088,-1052.87880,568.15560,185.63464,69.694440,4.080891,1.055449,0.001230,0.000015,22.888725,21.294197,20.550278,19.783850,20.024588,20.789160,NaN,16.100863,40.421947,49.645070,24.602830,10.031362,0.169507,-4.144490,PSF,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,58194.394229,57464.288310,2018-03-17 09:27:41.402,2016-03-17 06:55:10.000,9011_613761_421
608402,9011,2730,615266,164.752311,59.366784,0.108117,0.318219,0.909724,13.723277,7.506045,35.774296,-1355.27610,769.61080,237.15010,99.968710,3.639484,0.973910,0.001298,0.000016,24.885160,23.722904,22.591383,19.654633,20.310413,18.615847,NaN,2.999358,4.900465,9.095812,26.180490,7.407480,1.288823,-5.403758,PSF,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,57895.174033,57809.421548,2017-05-22 04:10:36.439,2017-02-25 10:07:01.770,9011_615266_2730
2250376,9011,3564,604361,155.673808,55.733950,0.228372,0.208837,0.308774,0.476354,1.097808,39.316856,-513.57275,661.72990,193.75407,149.976930,4.451325,1.084653,0.001300,0.000017,24.070723,24.178476,23.763588,23.303305,22.397535,18.513308,NaN,5.874658,2.906912,3.781401,1.005020,1.143331,1.417757,-2.093747,PSF,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,58226.262110,57784.409012,2018-04-18 06:17:26.310,2017-01-31 09:48:58.639,9011_604361_3564
2357573,9011,3761,601900,158.284790,55.109023,0.251771,0.937185,1.696882,-0.123897,0.892611,-16.660126,-142.61873,448.78406,178.64737,56.407830,4.552381,1.082338,0.001410,0.000017,23.972832,22.553827,21.916580,NaN,22.622477,NaN,NaN,5.333645,12.526324,12.744456,-0.264351,0.928633,-0.625595,-0.593522,REX,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,57901.173808,57518.158623,2017-05-28 04:10:17.000,2016-05-10 03:48:25.000,9011_601900_3761
2048095,9011,3208,609931,154.515359,57.599980,0.220955,0.239739,-0.050944,0.429848,1.463715,37.489258,-299.11280,677.59220,215.77026,126.830765,4.418661,1.041095,0.001279,0.000016,24.120808,24.038239,NaN,23.415657,22.085710,18.565094,NaN,5.751597,3.521553,-0.573730,0.903567,1.493488,1.340808,-1.214538,PSF,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,58194.386955,57787.376806,2018-03-17 09:17:12.879,2017-02-03 09:02:36.000,9011_609931_3208


# CSC

In [5]:
csc_df = pd.read_pickle(data_path+'csc_all_lh.pkl')
csc_df.sample(5)

,name,ra,dec,err_ellipse_r0,err_ellipse_r1,err_ellipse_ang,significance,likelihood,likelihood_class,conf_flag,dither_warning_flag,extent_flag,pileup_flag,sat_src_flag,streak_src_flag,var_flag,flux_aper_s,flux_aper_lolim_s,flux_aper_hilim_s,flux_aper_m,flux_aper_lolim_m,flux_aper_hilim_m,radec_err,r_98,flux_05_2,flux_05_2_err,secure
127161,2CXO J105458.6+572828,163.744343,57.474458,1.379133,1.207802,39.516309,2.054054,106.274514,TRUE,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,0.748953,2.094933,NaN,NaN,True
114855,2CXO J103429.9+573749,158.624646,57.630450,0.729089,0.726903,150.257204,5.562500,178.715113,TRUE,False,False,True,False,False,False,False,8.790114e-16,4.088425e-16,1.349180e-15,2.517898e-15,2.068273e-15,2.937548e-15,0.420608,1.176504,3.396910e-15,9.057515e-16,False
126956,2CXO J105357.5+571733,163.489939,57.292639,6.632219,5.590269,19.062279,2.702703,19.314616,TRUE,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,3.543647,9.912112,NaN,NaN,True
114512,2CXO J103251.7+542104,158.215615,54.351312,1.013930,1.013930,0.000000,4.125000,64.030474,TRUE,False,False,False,False,False,False,True,1.599604e-15,9.088659e-16,2.290342e-15,2.754934e-15,1.785606e-15,3.673246e-15,0.585810,1.638597,4.354538e-15,1.654428e-15,True
123302,2CXO J104636.1+584115,161.650471,58.687513,2.155024,1.198557,163.023810,2.594595,15.742410,MARGINAL,False,False,False,False,False,False,False,1.773937e-16,9.336512e-18,3.454510e-16,4.987223e-16,2.731098e-16,7.243347e-16,1.007417,2.817895,6.761160e-16,3.978548e-16,True


## CSC x ERO cross-match

In [6]:
ero_csc_orig = cross_match_data_frames(ero_df, csc_df,
 'RA_fin', 'DEC_fin', 'ra', 'dec', match_radius = 30, df_prefix = 'csc')
 
ero_csc = ero_csc_orig.query("csc_n_near==1 & csc_n_matches==1")

ero_csc = ero_csc.query("csc_secure==True")
print('SECURITY FILTER APPLIED (CSC)')
print('cross-matches: only one csc within 30 arcsec and it is unique for the eROSITA source', ero_csc.shape[0])
assert len(ero_csc) == ero_csc.csc_name.nunique()
ero_csc.sample(5)

cross-match radius 30 arcsec
total matches: 941 out of 9215 x 2329
	 total unique pairs: 868
	 total non-unique pairs (duplicates in df2): 73
SECURITY FILTER APPLIED (CSC)
cross-matches: only one csc within 30 arcsec and it is unique for the eROSITA source 555


,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,csc_name,csc_ra,csc_dec,csc_err_ellipse_r0,csc_err_ellipse_r1,csc_err_ellipse_ang,csc_significance,csc_likelihood,csc_likelihood_class,csc_conf_flag,csc_dither_warning_flag,csc_extent_flag,csc_pileup_flag,csc_sat_src_flag,csc_streak_src_flag,csc_var_flag,csc_flux_aper_s,csc_flux_aper_lolim_s,csc_flux_aper_hilim_s,csc_flux_aper_m,csc_flux_aper_lolim_m,csc_flux_aper_hilim_m,csc_radec_err,csc_r_98,csc_flux_05_2,csc_flux_05_2_err,csc_secure,csc_sep,csc_n_near,csc_n_matches
559,SRGe J104342.6+585331,160.927598,58.891865,148.859955,51.319811,6.396067,25.133575,6.948373e-15,1.585066e-15,32.475403,7.408302,4269.911621,0.0,0.0,0.0,5294,4320,160.929607,58.890575,2.731501,94.995201,0.000005,6.268413e+08,6.270701e+08,0.0,0.0,0.007606,0.001735,18.336561,160.927598,58.891865,-1,-1.000000,0,-1,NaN,-1.0,-1,CLANS 95,1.550,Galaxy,5.190417,2,1,0,0,16.212999,15.540000,12.498,18.9,11.9,0.8,1,1,0,1237658304889946558,0,7.013259,59293934489,2.731501,2.286637,3.451890,5.597109,134.582239,45.921571,5.347045e-15,7.468036e+19,2CXO J104342.8+585330,160.928746,58.891725,1.167996,1.017214,10.996690,3.696970,32.856541,TRUE,False,False,False,False,False,False,False,4.574946e-16,1.614687e-16,7.400648e-16,5.288385e-16,2.708685e-16,7.739100e-16,0.632766,1.769940,9.863331e-16,5.422968e-16,True,2.193647,1,1
331,SRGe J104604.6+584424,161.519056,58.739927,148.717363,51.649946,5.804560,53.263432,1.268957e-14,2.087466e-15,60.201984,9.903375,4334.224121,0.0,0.0,0.0,2454,1939,161.521064,58.738638,2.452672,131.308472,0.000006,6.268413e+08,6.270701e+08,0.0,0.0,0.013890,0.002285,18.390734,161.519056,58.739927,855293400709705088,9.511575,1,0,21.244097,-1.0,0,CLANS 397,1.810,QSO,9.511712,1,1,0,1,17.410999,15.957000,12.295,7.4,7.7,0.5,1,2,0,1237655109446467838,0,8.503062,59860040492,2.452672,2.075170,3.132660,5.079490,135.070783,45.959697,9.765120e-15,7.857937e+19,2CXO J104604.0+584424,161.516774,58.740109,0.865831,0.776664,15.442971,10.266414,543.170066,TRUE,False,False,False,False,False,False,False,6.272374e-15,5.221160e-15,7.265187e-15,4.759875e-15,4.055126e-15,5.464624e-15,0.475184,1.329160,1.103225e-14,1.756152e-15,True,4.312858,1,1
661,SRGe J105223.1+574123,163.096416,57.689667,149.074001,52.971169,7.358400,17.018959,5.822092e-15,1.494275e-15,27.135727,6.964548,4258.041504,0.0,0.0,0.0,4618,3731,163.098425,57.688378,3.179986,58.557049,0.000003,6.268413e+08,6.270701e+08,0.0,0.0,0.006373,0.001636,20.025534,163.096416,57.689667,854251441643925248,3.959439,1,0,20.773928,-1.0,0,2XMM J105223.3+574121,1.535,QSO,3.957990,1,1,0,0,17.440001,16.667999,12.461,8.3,4.3,0.8,1,1,1,1237655107836051961,0,3.963910,63840901581,3.179986,2.630678,3.971251,6.439233,136.887687,45.528919,4.480327e-15,6.988110e+19,2CXO J105223.3+574121,163.097384,57.689204,1.144663,0.873624,61.525188,2.342857,37.217880,TRUE,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,0.588278,1.645501,NaN,NaN,True,2.501893,1,1
424,SRGe J104919.6+572336,162.331593,57.393245,149.895352,52.865837,7.553347,32.012077,9.570218e-15,2.094181e-15,43.999744,9.628144,4200.256836,0.0,0.0,0.0,4239,3403,162.333602,57.391956,3.270256,92.757202,0.000004,6.268413e+08,6.270701e+08,0.0,0.0,0.010475,0.002292,22.633936,162.331593,57.393245,854220105563193344,4.767182,1,0,19.925060,-1.0,0,2XMM J104919.5+572334,1.682,QSO,4.773389,1,1,1,1,15.923000,14.575000,11.602,23.1,22.6,5.1,1,2,0,1237658302742528391,0,3.927156,64985857740,3.270256,2.700373,4.076461,6.609828,136.624053,45.057799,7.364656e-15,6.482540e+19,2CXO J1049

## CSC x DESI cross-match

In [7]:
csc_desi_all = cross_match_data_frames(ero_csc, desi, colname_ra1 = 'csc_ra',colname_dec1 = 'csc_dec' , colname_ra2 = 'ra', colname_dec2=  'dec', match_radius = 15, df_prefix = 'desi')
csc_desi_all

cross-match radius 15 arcsec
total matches: 2509 out of 555 x 2418574
	 total unique pairs: 2509
	 total non-unique pairs (duplicates in df2): 0


,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,csc_name,csc_ra,csc_dec,csc_err_ellipse_r0,csc_err_ellipse_r1,csc_err_ellipse_ang,csc_significance,csc_likelihood,csc_likelihood_class,csc_conf_flag,csc_dither_warning_flag,csc_extent_flag,csc_pileup_flag,csc_sat_src_flag,csc_streak_src_flag,csc_var_flag,csc_flux_aper_s,csc_flux_aper_lolim_s,csc_flux_aper_hilim_s,csc_flux_aper_m,csc_flux_aper_lolim_m,csc_flux_aper_hilim_m,csc_radec_err,csc_r_98,csc_flux_05_2,csc_flux_05_2_err,csc_secure,csc_sep,csc_n_near,csc_n_matches,desi_release,desi_objid,desi_brickid,desi_ra,desi_dec,desi_flux_g,desi_flux_r,desi_flux_z,desi_flux_w1,desi_flux_w2,desi_flux_w3,desi_flux_w4,desi_flux_ivar_g,desi_flux_ivar_r,desi_flux_ivar_z,desi_flux_ivar_w1,desi_flux_ivar_w2,desi_flux_ivar_w3,desi_flux_ivar_w4,desi_dered_mag_g,desi_dered_mag_r,desi_dered_mag_z,desi_dered_mag_w1,desi_dered_mag_w2,desi_dered_mag_w3,desi_dered_mag_w4,desi_snr_g,desi_snr_r,desi_snr_z,desi_snr_w1,desi_snr_w2,desi_snr_w3,desi_snr_w4,desi_type,desi_parallax,desi_parallax_ivar,desi_pmra,desi_pmra_ivar,desi_pmdec,desi_pmdec_ivar,desi_ref_cat,desi_ref_id,desi_mjd_max,desi_mjd_min,desi_iso_max,desi_iso_min,desi_desi_id,desi_sep,desi_n_near,desi_n_matches
2,SRGe J110242.6+594120,165.677384,59.688978,145.179791,52.486946,7.653812,211.181183,1.846204e-13,1.864554e-14,134.683884,13.602254,666.474426,0.0,0.0,0.0,679,556,165.679392,59.687689,3.316714,224.252457,0.000026,6.268413e+08,6.270701e+08,0.0,0.0,0.202084,0.020409,3.928642,165.677384,59.688978,860692071522063104,4.485168,1,0,17.027691,-1.0,0,6C 105938+595728,1.83,BLLac,4.485268,1,1,1,1,13.617000,12.659000,9.803,43.9,47.2,27.9,1,3,1,1237658304891060264,0,3.087692,56364004534,3.316714,2.736290,4.130682,6.697745,136.878443,47.936061,1.420726e-13,6.398279e+19,2CXO J110242.8+594122,165.678600,59.689447,2.291189,2.008557,31.648612,14.313177,629.754672,TRUE,False,False,False,False,False,False,False,1.247590e-13,1.097446e-13,1.397733e-13,9.484644e-14,8.480775e-14,1.048851e-13,1.244794,3.481875,2.196054e-13,2.554235e-14,True,2.779278,1,1,9011,4152,616737,165.678178,59.688774,123.783880,140.066150,194.811190,287.176500,388.446700,1089.463000,2591.550300,53.469475,22.336110,17.090487,1.092097,0.326052,0.001556,0.000019,17.247953,17.120436,16.768284,16.353460,16.025955,14.906816,13.966043,905.142700,661.967900,805.361940,300.109280,221.806760,42.980423,11.174195,PSF,-0.023313,140.88054,-0.008907,94.32313,0.07499,50.308243,G2,860692071522063104,58194.477145,57813.360879,2018-03-17 11:27:05.369,2017-03-01 08:39:39.978,9011_616737_4152,2.540902,7,1
1,SRGe J110242.6+594120,165.677384,59.688978,145.179791,52.486946,7.653812,211.181183,1.846204e-13,1.864554e-14,134.683884,13.602254,666.474426,0.0,0.0,0.0,679,556,165.679392,59.687689,3.316714,224.252457,0.000026,6.268413e+08,6.270701e+08,0.0,0.0,0.202084,0.020409,3.928642,165.677384,59.688978,860692071522063104,4.485168,1,0,17.027691,-1.0,0,6C 105938+595728,1.83,BLLac,4.485268,1,1,1,1,13.617000,12.659000,9.803,43.9,47.2,27.9,1,3,1,1237658304891060264,0,3.087692,56364004534,3.316714,2.736290,4.130682,6.697745,136.878443,47.936061,1.420726e-13,6.398279e+19,2CXO J110242.8+594122,165.678600,59.689447,2.291189,2.008557,31.648612,14.313177,629.754672,TRUE,False,False,False,False,False,False,False,1.247590e-13,1.097446e-13,1.397733e-13,9.484644e-14,8.480775e-14,1.048851e-13,1.244794,3.481875,2.196054e-13,2.554235e-14,True,2.779278,1,1,9011,4143,616737,165.676791,59.691063,-0.015934,0.072536,0.596225,3.677433,6.544262,-131.121550,-1138.750900,749.9

In [8]:
csc_desi_closest = (csc_desi_all
    .loc[csc_desi_all.groupby('csc_name')['desi_sep'].idxmin()]
    )

csc_hostless = csc_desi_closest.query("desi_sep>2*csc_r_98 & desi_sep>2*1.43")
#csc_hostless = csc_hostless.query('csc_sep<20') #why we have added this in the first place?

csc_hostless['csc_ero_flux_ratio'] = csc_hostless['csc_flux_05_2'] / csc_hostless['flux_05-20']


csc_hostless.rename(columns = {'csc_sep':'ero_csc_sep', 'desi_sep':'desi_sep_minimal'}, inplace = True)

csc_hostless = add_separation_columns(csc_hostless, 'RA_fin', 'DEC_fin', 'desi_ra', 'desi_dec', 'desi_ero_sep')

assert csc_hostless.srcname_fin.nunique()==csc_hostless.csc_name.nunique()
assert csc_hostless.srcname_fin.nunique()==csc_hostless.shape[0]


print('hostless csc near chandra sources', csc_hostless.shape[0])


csc_hostless.sample(7)

hostless csc near chandra sources 28


,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,csc_name,csc_ra,csc_dec,csc_err_ellipse_r0,csc_err_ellipse_r1,csc_err_ellipse_ang,csc_significance,csc_likelihood,csc_likelihood_class,csc_conf_flag,csc_dither_warning_flag,csc_extent_flag,csc_pileup_flag,csc_sat_src_flag,csc_streak_src_flag,csc_var_flag,csc_flux_aper_s,csc_flux_aper_lolim_s,csc_flux_aper_hilim_s,csc_flux_aper_m,csc_flux_aper_lolim_m,csc_flux_aper_hilim_m,csc_radec_err,csc_r_98,csc_flux_05_2,csc_flux_05_2_err,csc_secure,ero_csc_sep,csc_n_near,csc_n_matches,desi_release,desi_objid,desi_brickid,desi_ra,desi_dec,desi_flux_g,desi_flux_r,desi_flux_z,desi_flux_w1,desi_flux_w2,desi_flux_w3,desi_flux_w4,desi_flux_ivar_g,desi_flux_ivar_r,desi_flux_ivar_z,desi_flux_ivar_w1,desi_flux_ivar_w2,desi_flux_ivar_w3,desi_flux_ivar_w4,desi_dered_mag_g,desi_dered_mag_r,desi_dered_mag_z,desi_dered_mag_w1,desi_dered_mag_w2,desi_dered_mag_w3,desi_dered_mag_w4,desi_snr_g,desi_snr_r,desi_snr_z,desi_snr_w1,desi_snr_w2,desi_snr_w3,desi_snr_w4,desi_type,desi_parallax,desi_parallax_ivar,desi_pmra,desi_pmra_ivar,desi_pmdec,desi_pmdec_ivar,desi_ref_cat,desi_ref_id,desi_mjd_max,desi_mjd_min,desi_iso_max,desi_iso_min,desi_desi_id,desi_sep_minimal,desi_n_near,desi_n_matches,csc_ero_flux_ratio,desi_ero_sep
2051,SRGe J103520.2+573355,158.834253,57.565203,151.619002,51.302995,8.727764,10.466516,4.667627e-15,1.417703e-15,21.820000,6.627410,4270.769531,0.0,0.0,0.0,7244,6121,158.836262,57.563913,3.811093,153.681015,0.000007,6.268413e+08,6.270701e+08,0.0,0.0,0.005109,0.001552,19.799236,158.834253,57.565203,-1,-1.0,0,-1,NaN,-1.0,-1,CXCCLASXS J103520.9+573349,0.75,X,2.400142,1,1,0,1,17.680000,16.650999,12.528,6.3,4.4,0.7,1,0,-1,0,0,NaN,64320607603,3.811093,3.120235,4.710282,7.637545,134.251508,44.212833,3.591921e-15,5.738736e+19,2CXO J103520.9+573349,158.837140,57.563679,1.650398,1.276601,11.458660,3.875000,48.279513,TRUE,False,False,False,False,False,False,False,2.667747e-15,1.734035e-15,3.556996e-15,1.574239e-15,1.132347e-15,2.016130e-15,0.852421,2.384348,4.241985e-15,1.432870e-15,True,7.821427,1,1,9011,914,609941,158.835675,57.562204,0.110228,0.502531,1.554247,6.035648,6.055666,21.223442,513.94025,598.09480,147.184770,92.629630,4.490143,1.158340,0.001397,0.000018,24.876577,23.235174,22.014534,20.547176,20.543974,19.182827,15.722669,2.695733,6.096696,14.958735,12.789513,6.517482,0.793381,2.169543,REX,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000,NaN,0,58194.395665,57787.382199,2018-03-17 09:29:45.430,2017-02-03 09:10:22,9011_609941_914,6.015550,5,1,1.180980,11.139738
2507,SRGe J104241.6+591357,160.673352,59.232589,148.587833,50.997242,11.341792,6.555059,2.857708e-15,1.101090e-15,13.649315,5.259152,4363.555176,0.0,0.0,0.0,9310,8154,160.675361,59.231300,5.003250,139.718124,0.000008,6.268413e+08,6.270701e+08,0.0,0.0,0.003128,0.001205,17.367970,160.673352,59.232589,-1,-1.0,0,-1,NaN,-1.0,-1,SWIRE J104241.45+591357.2,NaN,X,7.028654,1,1,1,-1,NaN,NaN,NaN,NaN,NaN,NaN,0,0,-1,0,0,NaN,58033438718,5.003250,4.054768,6.121046,9.925044,134.158384,46.137393,2.199118e-15,8.289398e+19,2CXO J104241.3+591356,160.672263,59.232319,2.365459,1.212545,41.542077,3.937500,36.298269,TRUE,False,False,False,False,False,False,False,8.802631e-16,3.385627e-16,1.399393e-15,1.018344e-15,6.724912e-16,1.344982e-15,1.085950,3.037565,1.898607e-15,8.883926e-16,True,2.230656,1,1,9011,1096,615258,160.668637,59.233390,1.436928,5.050992,12.627905,35.036650,33.209656,289.941440,1063.59700,597.12630,214.927890,44.564034,3.310745,0.929113,0.000901,0.000012,22.081808,20.724985,19.

***

# 4XMM DR10

In [9]:
xmm_df = pd.read_pickle(data_path+'xmm_all_lh.pkl')
xmm_df.sample(5)

,srcid,iauname,sc_ra,sc_dec,sc_poserr,sc_det_ml,sc_ep_1_flux,sc_ep_1_flux_err,sc_ep_2_flux,sc_ep_2_flux_err,sc_ep_3_flux,sc_ep_3_flux_err,sc_ep_4_flux,sc_ep_4_flux_err,sc_ep_5_flux,sc_ep_5_flux_err,sc_ep_8_flux,sc_ep_8_flux_err,sc_ep_9_flux,sc_ep_9_flux_err,sc_ep_8_fmin,sc_ep_8_fmin_err,sc_ep_8_fmax,sc_ep_8_fmax_err,sc_hr1,sc_hr1_err,sc_hr2,sc_hr2_err,sc_hr3,sc_hr3_err,sc_hr4,sc_hr4_err,sc_extent,sc_ext_err,sc_ext_ml,sc_chi2prob,sc_var_flag,sc_sum_flag,mjd_first,mjd_last,sc_fvar,sc_fvarerr,n_detections,confused,webpage_url,flux_05_2,flux_05_2_err,sigma,r_98,secure
255084,201237001015035,4XMM J105216.9+572017,163.070467,57.338119,0.163175,986.8180,5.206270e-17,2.907080e-17,5.262170e-16,5.987130e-17,7.337220e-15,1.841540e-16,1.860800e-14,5.039670e-16,4.344790e-14,2.538020e-15,7.099190e-14,2.610490e-15,2.014150e-14,3.682520e-16,4.256790e-14,1.268490e-14,1.020470e-13,9.314970e-15,0.804502,0.058241,0.854036,0.014845,-0.053515,0.017979,-0.185125,0.027850,0.0,NaN,0.191753,0.034727,False,1,51661.149132,52615.192419,1.575233,0.648893,16,False,http://xmm-catalog.irap.omp.eu/source/20123700...,7.863437e-15,1.936421e-16,0.115715,0.323673,True
255968,201475111010040,4XMM J105453.9+573137,163.724857,57.527174,0.464717,188.8720,3.770500e-15,3.708460e-16,3.208420e-15,3.375730e-16,3.511570e-15,3.772960e-16,2.718570e-15,7.004180e-16,1.885180e-15,2.170530e-15,1.559570e-14,2.480790e-15,1.167300e-14,8.229430e-16,1.395050e-14,4.589300e-15,8.008980e-14,4.658390e-14,-0.035434,0.066432,-0.016788,0.071890,-0.502584,0.090781,-0.422917,0.210733,0.0,NaN,-0.877501,0.213683,False,0,52570.341921,55297.794479,0.317537,0.343142,4,False,http://xmm-catalog.irap.omp.eu/source/20147511...,6.719990e-15,5.062685e-16,0.329553,0.921809,True
252017,205541201010045,4XMM J104539.4+591225,161.414365,59.207149,0.823742,82.7593,2.078130e-15,4.550340e-16,4.318920e-15,6.043260e-16,3.595170e-15,6.667090e-16,2.307140e-15,1.112810e-15,3.281760e-15,6.814850e-15,2.186390e-14,7.359140e-15,1.376410e-14,1.515740e-15,1.805180e-14,1.232230e-14,4.210980e-14,2.503990e-14,0.262863,0.098191,-0.165955,0.107476,-0.411764,0.140427,0.285417,0.195835,0.0,NaN,-0.799177,0.968214,False,0,54750.880949,54781.372685,NaN,NaN,3,False,http://xmm-catalog.irap.omp.eu/source/20554120...,7.914090e-15,8.998393e-16,0.584155,1.633968,True
255429,201237001010023,4XMM J105302.4+573756,163.260099,57.632431,0.163899,919.1000,3.653040e-15,1.229340e-16,4.831330e-15,1.400280e-16,5.264520e-15,1.629310e-16,6.910480e-15,3.491750e-16,1.169560e-14,1.678170e-15,3.477840e-14,1.799300e-15,1.874210e-14,3.618270e-16,1.721490e-14,8.386370e-15,5.416550e-14,3.671420e-14,0.125091,0.021045,-0.003268,0.020425,-0.357984,0.024483,-0.322627,0.051177,0.0,NaN,-0.655994,0.019178,False,0,51661.149132,52615.192419,0.801261,0.275276,16,False,http://xmm-catalog.irap.omp.eu/source/20123700...,1.009585e-14,2.148356e-16,0.116229,0.325109,True
252592,205562116010030,4XMM J104710.6+572602,161.794274,57.434084,2.345730,12.5642,1.073290e-15,6.397700e-16,1.803170e-15,8.119960e-16,5.258180e-16,5.478150e-16,2.832510e-15,1.940190e-15,1.356710e-14,1.446560e-14,2.765880e-14,1.499590e-14,8.746030e-15,2.225050e-15,2.765880e-14,1.499590e-14,2.765880e-14,1.499590e-14,0.085560,0.331617,-0.237185,0.271746,0.401811,0.193991,-0.438294,0.377594,0.0,NaN,0.866945,NaN,NaN,0,54799.044537,54799.125949,NaN,NaN,1,False,http://xmm-catalog.irap.omp.eu/source/20556211...,2.328988e-15,9.795095e-16,1.663469,4.652972,True


## XMM x eROSITA cross-match

In [10]:
ero_xmm_orig = cross_match_data_frames(ero_df, xmm_df,
 'RA_fin', 'DEC_fin', 'sc_ra', 'sc_dec', match_radius = 30, df_prefix = 'xmm')
 
ero_xmm = ero_xmm_orig.query("xmm_n_near==1 & xmm_n_matches==1")
ero_xmm = ero_xmm.query("xmm_secure==True")
print('SECURITY FILTER APPLIED (XMM)')

ero_xmm = ero_xmm.query("xmm_secure==True")
print('cross-matches: only one xmm within 30 arcsec and it is unique for the eROSITA source', ero_xmm.shape[0])
assert len(ero_xmm) == ero_xmm.xmm_srcid.nunique()
ero_xmm.sample(5)

cross-match radius 30 arcsec
total matches: 978 out of 9215 x 1791
	 total unique pairs: 944
	 total non-unique pairs (duplicates in df2): 34
SECURITY FILTER APPLIED (XMM)
cross-matches: only one xmm within 30 arcsec and it is unique for the eROSITA source 742


,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,xmm_srcid,xmm_iauname,xmm_sc_ra,xmm_sc_dec,xmm_sc_poserr,xmm_sc_det_ml,xmm_sc_ep_1_flux,xmm_sc_ep_1_flux_err,xmm_sc_ep_2_flux,xmm_sc_ep_2_flux_err,xmm_sc_ep_3_flux,xmm_sc_ep_3_flux_err,xmm_sc_ep_4_flux,xmm_sc_ep_4_flux_err,xmm_sc_ep_5_flux,xmm_sc_ep_5_flux_err,xmm_sc_ep_8_flux,xmm_sc_ep_8_flux_err,xmm_sc_ep_9_flux,xmm_sc_ep_9_flux_err,xmm_sc_ep_8_fmin,xmm_sc_ep_8_fmin_err,xmm_sc_ep_8_fmax,xmm_sc_ep_8_fmax_err,xmm_sc_hr1,xmm_sc_hr1_err,xmm_sc_hr2,xmm_sc_hr2_err,xmm_sc_hr3,xmm_sc_hr3_err,xmm_sc_hr4,xmm_sc_hr4_err,xmm_sc_extent,xmm_sc_ext_err,xmm_sc_ext_ml,xmm_sc_chi2prob,xmm_sc_var_flag,xmm_sc_sum_flag,xmm_mjd_first,xmm_mjd_last,xmm_sc_fvar,xmm_sc_fvarerr,xmm_n_detections,xmm_confused,xmm_webpage_url,xmm_flux_05_2,xmm_flux_05_2_err,xmm_sigma,xmm_r_98,xmm_secure,xmm_sep,xmm_n_near,xmm_n_matches
738,SRGe J104633.8+574056,161.640791,57.682351,149.939341,52.396427,14.710700,10.380621,5.778930e-15,1.671769e-15,27.221912,7.874943,4303.469238,0.0,0.0,0.0,5901,4865,161.642800,57.681062,6.527811,59.763466,0.000006,6.268413e+08,6.270701e+08,0.0,0.0,0.006326,0.001830,72.756439,161.640791,57.682351,854435644201818240,2.617377,1,0,21.507118,-1.0,0,2MASS J10463392+5741039,NaN,Star,12.289124,1,1,0,-1,NaN,NaN,NaN,NaN,NaN,NaN,0,2,0,1237658303279268689,0,2.571851,63868773154,6.527811,5.259175,7.939210,12.873129,135.961319,45.103285,4.447112e-15,6.784015e+19,205562116010040,4XMM J104634.0+574103,161.641798,57.684366,1.569880,17.4237,1.886530e-16,6.421310e-16,7.586620e-15,2.203530e-15,1.365160e-15,1.828880e-15,8.829180e-15,5.168940e-15,1.687110e-14,2.529950e-14,3.828010e-14,2.623480e-14,2.150120e-14,5.149970e-15,3.828010e-14,2.623480e-14,3.828010e-14,2.623480e-14,0.930490,0.136991,-0.453266,0.274613,-0.177993,0.391506,-0.351841,0.598087,0.0,NaN,-0.735788,NaN,NaN,1,54799.044537,54799.125949,NaN,NaN,1,False,http://xmm-catalog.irap.omp.eu/source/20556211...,8.951780e-15,2.863625e-15,1.113277,3.114002,True,7.508886,1,1
56,SRGe J103435.1+594936,158.646126,59.826701,148.914687,49.828402,4.461424,182.111481,5.059594e-14,5.601032e-15,101.672218,11.255239,1835.836914,0.0,0.0,0.0,819,663,158.648135,59.825411,1.803823,195.232910,0.000014,6.268413e+08,6.270701e+08,0.0,0.0,0.055382,0.006131,9.024734,158.646126,59.826701,1047751572438736256,5.803524,1,1,14.356899,NaN,0,2XMM J103435.0+594937,NaN,PM*,6.121909,1,1,0,0,11.140,11.097,10.958,47.8,51.9,11.5,1,1,1,1237655369819291758,0,6.056801,55865502667,1.803823,1.594990,2.407784,3.904130,132.447447,46.085187,3.893555e-14,8.874502e+19,203060507010008,4XMM J103435.1+594936,158.646539,59.826844,0.961796,240.5540,1.172890e-14,2.015850e-15,1.949790e-14,2.523210e-15,9.831610e-15,2.070710e-15,5.453910e-16,1.830310e-15,0.000000e+00,1.648410e-14,4.327140e-14,1.704540e-14,4.616730e-14,5.210940e-15,4.327140e-14,1.704540e-14,4.327140e-14,1.704540e-14,0.282463,0.096779,-0.432981,0.095748,-0.939067,0.125189,-1.000000,3.960440,0.0,NaN,-1.751450,NaN,NaN,0,53464.316447,53464.431447,NaN,NaN,1,False,http://xmm-catalog.irap.omp.eu/source/20306050...,2.932951e-14,3.264112e-15,0.682055,1.907811,True,0.908176,1,1
256,SRGe J104048.7+590623,160.202865,59.106443,148.989384,50.896998,5.329939,82.591988,1.687896e-14,2.346200e-15,80.034996,11.124984,4331.934082,0.0,0.0,0.0,2193,1723,160.204874,59.105153,2.226454,66.872421,0.000007,6.268413e+08,6.270701e+08,0.0,0.0,0.018476,0.002568,17.580353,160.202865,59.106443,855469318274772480,5.927181,1,0,19.392134,-1.0,0,[VV2006] J104049.1+590624,1.588040,QSO,5.924814,1,1,1,1,16.160,14.896,11

## XMM x DESI cross-match

In [11]:
xmm_desi_all = cross_match_data_frames(ero_xmm, desi, colname_ra1 = 'xmm_sc_ra',colname_dec1 = 'xmm_sc_dec' , colname_ra2 = 'ra', colname_dec2=  'dec', match_radius = 15, df_prefix = 'desi')
xmm_desi_all

cross-match radius 15 arcsec
total matches: 3293 out of 742 x 2418574
	 total unique pairs: 3293
	 total non-unique pairs (duplicates in df2): 0


,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,xmm_srcid,xmm_iauname,xmm_sc_ra,xmm_sc_dec,xmm_sc_poserr,xmm_sc_det_ml,xmm_sc_ep_1_flux,xmm_sc_ep_1_flux_err,xmm_sc_ep_2_flux,xmm_sc_ep_2_flux_err,xmm_sc_ep_3_flux,xmm_sc_ep_3_flux_err,xmm_sc_ep_4_flux,xmm_sc_ep_4_flux_err,xmm_sc_ep_5_flux,xmm_sc_ep_5_flux_err,xmm_sc_ep_8_flux,xmm_sc_ep_8_flux_err,xmm_sc_ep_9_flux,xmm_sc_ep_9_flux_err,xmm_sc_ep_8_fmin,xmm_sc_ep_8_fmin_err,xmm_sc_ep_8_fmax,xmm_sc_ep_8_fmax_err,xmm_sc_hr1,xmm_sc_hr1_err,xmm_sc_hr2,xmm_sc_hr2_err,xmm_sc_hr3,xmm_sc_hr3_err,xmm_sc_hr4,xmm_sc_hr4_err,xmm_sc_extent,xmm_sc_ext_err,xmm_sc_ext_ml,xmm_sc_chi2prob,xmm_sc_var_flag,xmm_sc_sum_flag,xmm_mjd_first,xmm_mjd_last,xmm_sc_fvar,xmm_sc_fvarerr,xmm_n_detections,xmm_confused,xmm_webpage_url,xmm_flux_05_2,xmm_flux_05_2_err,xmm_sigma,xmm_r_98,xmm_secure,xmm_sep,xmm_n_near,xmm_n_matches,desi_release,desi_objid,desi_brickid,desi_ra,desi_dec,desi_flux_g,desi_flux_r,desi_flux_z,desi_flux_w1,desi_flux_w2,desi_flux_w3,desi_flux_w4,desi_flux_ivar_g,desi_flux_ivar_r,desi_flux_ivar_z,desi_flux_ivar_w1,desi_flux_ivar_w2,desi_flux_ivar_w3,desi_flux_ivar_w4,desi_dered_mag_g,desi_dered_mag_r,desi_dered_mag_z,desi_dered_mag_w1,desi_dered_mag_w2,desi_dered_mag_w3,desi_dered_mag_w4,desi_snr_g,desi_snr_r,desi_snr_z,desi_snr_w1,desi_snr_w2,desi_snr_w3,desi_snr_w4,desi_type,desi_parallax,desi_parallax_ivar,desi_pmra,desi_pmra_ivar,desi_pmdec,desi_pmdec_ivar,desi_ref_cat,desi_ref_id,desi_mjd_max,desi_mjd_min,desi_iso_max,desi_iso_min,desi_desi_id,desi_sep,desi_n_near,desi_n_matches
5,SRGe J104117.7+584929,160.323629,58.824642,149.266632,51.126914,2.166339,3391.888916,2.361506e-13,7.471780e-15,1120.011108,35.437035,4332.920898,0.0,0.0,0.0,38,27,160.325638,58.823353,0.454968,97.643723,0.000006,6.268413e+08,6.270701e+08,0.0,0.0,0.258489,0.008179,18.102430,160.323629,58.824642,-1,-1.0,0,-1,NaN,-1.0,-1,NaN,NaN,NaN,-1.000000,0,1,1,0,15.243,15.129000,12.821,31.5,15.9,-0.2,1,3,0,1237658304889815429,0,2.181367,59543516327,0.454968,0.774481,1.169150,1.895733,134.256532,45.696772,1.817271e-13,7.704875e+19,205562112010003,4XMM J104117.9+584929,160.324685,58.824762,0.817968,742.5780,2.808530e-14,3.382310e-15,4.008580e-14,4.210150e-15,4.761820e-14,4.978040e-15,5.426170e-14,9.520310e-15,8.843220e-15,2.233210e-14,1.909300e-13,2.733320e-14,1.567980e-13,1.062820e-14,1.909300e-13,2.733320e-14,1.909300e-13,2.733320e-14,0.136026,0.075853,0.023020,0.072849,-0.437121,0.081245,-0.896701,0.157718,0.0,NaN,-2.276850,0.425014,False,0,54595.266366,54595.352928,0.012697,1.426367,1,False,http://xmm-catalog.irap.omp.eu/source/20556211...,8.770400e-14,6.519681e-15,0.580060,1.622515,True,2.015272,1,1,9011,43,613766,160.324219,58.824639,-0.052349,0.298325,1.900912,2.352417,2.387243,53.370483,2019.365800,671.00964,212.87097,70.456120,2.689567,0.915560,0.001440,0.000018,NaN,23.799402,21.794834,21.570036,21.554535,18.181593,14.236904,-1.356040,4.352592,15.955904,3.857940,2.284231,2.025409,8.467377,PSF,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,58194.398524,57851.315637,2018-03-17 09:33:52.450,2017-04-08 07:34:31,9011_613766_43,0.975167,9,1
6,SRGe J104117.7+584929,160.323629,58.824642,149.266632,51.126914,2.166339,3391.888916,2.361506e-13,7.471780e-15,1120.011108,35.437035,4332.920898,0.0,0.0,0.0,38,27,160.325638,58.823353,0.454968,97.643723,0.000006,6.268413e+08,6.270701e+08,0.0,0.0,0.258489,0.008179,18.102430,160.323629,58.824642,-1,-1.0,0,-1,NaN,-1.0,-1,NaN,NaN,NaN,-1.000000,0,1,1,0,15.243,15.129000,12.821,31.5,15.9,-0.2,1,3,0,1237658304889815429,0,2.181367,59543516327,0.4549

In [12]:
xmm_desi_closest = (xmm_desi_all
    .loc[xmm_desi_all.groupby('xmm_srcid')['desi_sep'].idxmin()]
    )

xmm_hostless = xmm_desi_closest.query("desi_sep>2*xmm_r_98 & desi_sep>2*1.43")
#xmm_hostless = xmm_hostless.query('xmm_sep<20') #why we have added this in the first place?

xmm_hostless['xmm_ero_flux_ratio'] = xmm_hostless['xmm_flux_05_2'] / xmm_hostless['flux_05-20']


xmm_hostless.rename(columns = {'xmm_sep':'ero_xmm_sep', 'desi_sep':'desi_sep_minimal'}, inplace = True)

xmm_hostless = add_separation_columns(xmm_hostless, 'RA_fin', 'DEC_fin', 'desi_ra', 'desi_dec', 'desi_ero_sep')

assert xmm_hostless.srcname_fin.nunique()==xmm_hostless.xmm_srcid.nunique()
assert xmm_hostless.srcname_fin.nunique()==xmm_hostless.shape[0]


print('hostless xmm near chandra sources', xmm_hostless.shape[0])


xmm_hostless.sample(7)

hostless xmm near chandra sources 33


,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,xmm_srcid,xmm_iauname,xmm_sc_ra,xmm_sc_dec,xmm_sc_poserr,xmm_sc_det_ml,xmm_sc_ep_1_flux,xmm_sc_ep_1_flux_err,xmm_sc_ep_2_flux,xmm_sc_ep_2_flux_err,xmm_sc_ep_3_flux,xmm_sc_ep_3_flux_err,xmm_sc_ep_4_flux,xmm_sc_ep_4_flux_err,xmm_sc_ep_5_flux,xmm_sc_ep_5_flux_err,xmm_sc_ep_8_flux,xmm_sc_ep_8_flux_err,xmm_sc_ep_9_flux,xmm_sc_ep_9_flux_err,xmm_sc_ep_8_fmin,xmm_sc_ep_8_fmin_err,xmm_sc_ep_8_fmax,xmm_sc_ep_8_fmax_err,xmm_sc_hr1,xmm_sc_hr1_err,xmm_sc_hr2,xmm_sc_hr2_err,xmm_sc_hr3,xmm_sc_hr3_err,xmm_sc_hr4,xmm_sc_hr4_err,xmm_sc_extent,xmm_sc_ext_err,xmm_sc_ext_ml,xmm_sc_chi2prob,xmm_sc_var_flag,xmm_sc_sum_flag,xmm_mjd_first,xmm_mjd_last,xmm_sc_fvar,xmm_sc_fvarerr,xmm_n_detections,xmm_confused,xmm_webpage_url,xmm_flux_05_2,xmm_flux_05_2_err,xmm_sigma,xmm_r_98,xmm_secure,ero_xmm_sep,xmm_n_near,xmm_n_matches,desi_release,desi_objid,desi_brickid,desi_ra,desi_dec,desi_flux_g,desi_flux_r,desi_flux_z,desi_flux_w1,desi_flux_w2,desi_flux_w3,desi_flux_w4,desi_flux_ivar_g,desi_flux_ivar_r,desi_flux_ivar_z,desi_flux_ivar_w1,desi_flux_ivar_w2,desi_flux_ivar_w3,desi_flux_ivar_w4,desi_dered_mag_g,desi_dered_mag_r,desi_dered_mag_z,desi_dered_mag_w1,desi_dered_mag_w2,desi_dered_mag_w3,desi_dered_mag_w4,desi_snr_g,desi_snr_r,desi_snr_z,desi_snr_w1,desi_snr_w2,desi_snr_w3,desi_snr_w4,desi_type,desi_parallax,desi_parallax_ivar,desi_pmra,desi_pmra_ivar,desi_pmdec,desi_pmdec_ivar,desi_ref_cat,desi_ref_id,desi_mjd_max,desi_mjd_min,desi_iso_max,desi_iso_min,desi_desi_id,desi_sep_minimal,desi_n_near,desi_n_matches,xmm_ero_flux_ratio,desi_ero_sep
3265,SRGe J105729.0+573922,164.370932,57.656079,148.328791,53.487539,9.473058,7.259020,3.200342e-15,1.232167e-15,12.649606,4.870237,3611.005371,0.0,0.0,0.0,9168,8013,164.372941,57.654789,4.152283,69.306076,0.000007,6.268413e+08,6.270701e+08,0.0,0.0,0.003503,0.001349,30.617397,164.370932,57.656079,-1,-1.0,0,-1,NaN,-1.0,-1,NaN,NaN,NaN,-1.000000,0,1,0,-1,NaN,NaN,NaN,NaN,NaN,NaN,0,0,-1,0,0,NaN,63970321614,4.152283,3.386683,5.112509,8.289742,137.731073,45.873429,2.462788e-15,6.548207e+19,206060301010068,4XMM J105729.3+573919,164.372405,57.655527,1.496010,18.3006,7.492550e-16,3.277770e-16,1.229040e-15,4.277190e-16,1.230730e-15,5.336830e-16,8.787400e-16,9.783700e-16,1.035290e-15,5.706280e-15,9.560510e-15,6.112530e-15,5.093190e-15,1.217570e-15,9.560510e-15,6.112530e-15,9.560510e-15,6.112530e-15,0.385222,0.217221,-0.158138,0.240974,-0.204565,0.270570,-0.932835,0.343205,0.0,NaN,-0.750795,NaN,NaN,0,55121.968056,55122.491007,NaN,NaN,1,False,http://xmm-catalog.irap.omp.eu/source/20606030...,2.459770e-15,6.839306e-16,1.060892,2.967474,True,3.462272,1,1,9011,1790,610728,164.366732,57.656110,1.679582,3.104448,4.405715,1.793063,0.725399,-8.470533,-1280.23410,274.96027,110.016075,46.500250,4.322303,1.073355,0.000997,0.000013,21.912622,21.253618,20.880774,21.864616,22.847702,NaN,NaN,27.850697,32.562107,30.043055,3.727805,0.751534,-0.267492,-4.542855,EXP,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,58194.408691,57737.529606,2018-03-17 09:48:30.941,2016-12-15 12:42:38.000,9011_610728_1790,11.126276,6,1,0.998775,8.090106
965,SRGe J104620.0+572511,161.583130,57.419656,150.299295,52.544364,5.237901,75.854263,1.618196e-14,2.332513e-15,74.646103,10.759703,4214.282227,0.0,0.0,0.0,1139,917,161.585138,57.418366,2.182265,61.742287,0.000007,6.268413e+08,6.270701e+08,0.0,0.0,0.017713,0.002553,18.370316,161.583130,57.419656,-1,-1.0,0,-1,NaN,-1.0,-1,NaN,NaN,NaN,-1.000000,0,1,0,-1,NaN,NaN,NaN,NaN,NaN,NaN,0,0,-1,0,0,NaN,64883510054,2.182265,1.872585,2.82684

# Joining CSC and XMM counterparts

In [13]:
csc_ctps = csc_hostless[['srcname_fin', 'RA_fin','DEC_fin', 'DET_LIKE_0', 'pos_r98', 'pos_sigma_2d', 'flux_05-20', 'ero_csc_sep', 'csc_name',   'csc_ra', 'csc_dec', 'csc_flux_05_2', 'csc_r_98', 'desi_sep_minimal', 'desi_desi_id',  'desi_ra', 'desi_dec', 'csc_ero_flux_ratio']]

xmm_ctps = xmm_hostless[['srcname_fin', 'RA_fin','DEC_fin', 'DET_LIKE_0', 'pos_r98', 'pos_sigma_2d', 'flux_05-20', 'ero_xmm_sep', 'xmm_srcid',   'xmm_sc_ra', 'xmm_sc_dec', 'xmm_flux_05_2', 'xmm_r_98', 'desi_sep_minimal', 'desi_desi_id',  'desi_ra', 'desi_dec', 'xmm_ero_flux_ratio']]

csc_ctps.rename(columns = {'desi_desi_id':'desi_id'}, inplace = True)
xmm_ctps.rename(columns = {'desi_desi_id':'desi_id'}, inplace = True)

In [14]:
print('CSC companions', csc_ctps.shape[0])
print('XMM companions', xmm_ctps.shape[0])
print('Total companions possible', xmm_ctps.shape[0] + csc_ctps.shape[0])

XMM companions 33
CSC companions 28
Total companions possible 61


In [15]:
xmm_ctps = xmm_ctps[xmm_ctps['xmm_ero_flux_ratio'] <  5 ]
xmm_ctps = xmm_ctps[xmm_ctps['xmm_ero_flux_ratio'] >  1/5 ]

csc_ctps = csc_ctps[csc_ctps['csc_ero_flux_ratio'] <  5 ]
csc_ctps = csc_ctps[csc_ctps['csc_ero_flux_ratio'] >  1/5 ]

In [16]:
xmm_ctps = xmm_ctps[['srcname_fin', 'RA_fin', 'DEC_fin', 'pos_sigma_2d', 'flux_05-20', 'DET_LIKE_0', 'desi_id', 'desi_ra', 'desi_dec']]
csc_ctps = csc_ctps[['srcname_fin', 'RA_fin', 'DEC_fin', 'pos_sigma_2d', 'flux_05-20', 'DET_LIKE_0','desi_id', 'desi_ra', 'desi_dec']]
xmm_ctps['x_ray_det'] = 'xmm'
csc_ctps['x_ray_det'] = 'csc'
print('After Flux filter')
print('CSC companions', csc_ctps.shape[0])
print('XMM companions', xmm_ctps.shape[0])
print('Total companions possible', xmm_ctps.shape[0] + csc_ctps.shape[0])

After Flux filter
XMM companions 31
CSC companions 20
Total companions possible 51


In [21]:
final_ctps = pd.concat([xmm_ctps, csc_ctps])
final_ctps.sort_values(by='srcname_fin', inplace=True)
final_ctps.reset_index(drop=True, inplace=True)


n_ctps = final_ctps.groupby('srcname_fin')['x_ray_det'].transform(lambda x: len(x.unique())) 
final_ctps['n_x_ray_det'] = n_ctps

n_desi_ctps = final_ctps.groupby('srcname_fin')['desi_id'].transform(lambda x: len(x.unique()))
final_ctps['n_desi_ctps'] = n_desi_ctps


final_ctps = final_ctps.query('( n_x_ray_det == 1 ) | ( n_x_ray_det==2 & n_desi_ctps == 1 )')
## final_ctps.query('( n_x_ray_det == 1 ) | ( n_x_ray_det==2 & n_desi_ctps == 1 )') - opposite of this


final_ctps.drop_duplicates(subset=['srcname_fin', 'desi_id'], inplace=True)


final_ctps = add_separation_columns(final_ctps, colname_ra1 = 'RA_fin', colname_dec1 = 'DEC_fin', colname_ra2 = 'desi_ra', colname_dec2 = 'desi_dec', colname = 'sep_ero_desi')



# #save final_ctps_csv to csv
final_ctps_csv = final_ctps[['srcname_fin', 'desi_id']]
final_ctps_csv.rename(columns={'desi_id':'desi_id_true'}, inplace=True)
final_ctps_csv.desi_id_true = 'hostless'
final_ctps_csv.to_csv(data_path+'validation_ctps_ero_desi_lh_hostless.csv', index=False)
print('Final number of hostless', final_ctps.shape[0])

#final_ctps

Final number of hostless 48


In [22]:
#combine with non-hostless list
df_ctps = pd.read_csv(data_path+'validation_ctps_ero_desi_lh_no_hostless.csv')
df_hostless = pd.read_csv(data_path+'validation_ctps_ero_desi_lh_hostless.csv')
df_ctps = pd.concat([df_ctps, df_hostless])

df_ctps.to_csv(data_path+'validation_ctps_ero_desi_lh.csv', index = False)
df_ctps.sample(10)

,srcname_fin,desi_id_true
150,SRGe J103759.9+580931,9011_612251_2337
100,SRGe J103436.1+601012,9011_618172_1469
163,SRGe J103849.6+584306,9011_613764_2664
537,SRGe J105407.1+573523,9011_609951_1582
463,SRGe J105021.8+591145,9011_615262_755
495,SRGe J105131.4+551928,9011_602738_3251
96,SRGe J103432.7+582413,9011_613009_591
38,SRGe J103314.0+601347,9011_618171_2583
135,SRGe J103559.3+581016,9011_612250_2067
519,SRGe J105305.6+573426,9011_609950_3459
